<a href="https://colab.research.google.com/github/saianil58/NLP/blob/main/preprocessing_after_performance_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install ipython-autotime
%load_ext autotime

time: 209 µs (started: 2021-09-28 16:46:36 +00:00)


In [3]:
!pip install spacy
!pip install spacy_fastlang
!python3 -m spacy download en_core_web_sm
!pip install talon

     |████████████████████████████████| 13.6 MB 69 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
time: 20 s (started: 2021-09-28 16:46:36 +00:00)


In [4]:
import pandas as pd
import numpy as np
import re
import dateutil.parser as dparser
import spacy
import warnings
warnings.filterwarnings("ignore")
import spacy_fastlang
import talon
from talon import quotations

talon.init()

time: 3.58 s (started: 2021-09-28 16:46:56 +00:00)


In [5]:
# !pip install langdetect 
from langdetect import detector_factory
from langdetect import detect


time: 12.8 ms (started: 2021-09-28 16:46:59 +00:00)


In [15]:
def msg_parse(df):
    parse_message_date = ''
    try:
        parse_message_date = pd.to_datetime(df['MessageDate']) #python datetime object
    except Exception as e:
        print("parse error in Date")
    df['MessageDate'] = parse_message_date 
    
#     df = df[df['Inbound Emails'] <= 3]
    df = df.sort_values('MessageDate', ascending=False)
    df = df.drop_duplicates(subset='CaseNumber', keep='last')
    df = df.reset_index()
    df = df.drop(['index'], axis = 1) 
    return df



def read_csv_file1(file):
    j=0
    df = pd.read_csv(file,encoding = "ISO-8859-1")
    #df=pd.read_csv(io.BytesIO(uploaded['raw_train_data.csv']),encoding = "ISO-8859-1")
    df = df[df.columns[0:8]]
    dfpr = pd.DataFrame(columns=df.columns[0:8],index=range(df.shape[0]))
    
    for item in df.columns:
        df[item]=df[item].replace(np.nan,'',regex = True)
        
    print(df.shape[0])
    
    for i in range(df.shape[0]):
      if 1 == 0 and df['Id'][i] == '' and df['sub_type__c'][i] == '':
            print('if ', i)
            dfpr.loc[j][4] = df.loc[i][4]
            
            mail = df['TextBody'][i]
            while i < df.shape[0]:
                i = i + 1
                #print(i)
                #print(df['Case Number'][i])
                if df['Id'][i] == '':
                    break
                else:
                    if df.loc[i][0]!='' and df.loc[i][1]!='' and df.loc[i][2]!='':
                        print('insert ', i)
                        mail = mail + str(df.loc[i][0])
                        if df['Id'][i] != '':
                            print('date ', i)
                            dfpr.loc[j][1] = mail
                            dfpr.loc[j][2:5] = df.loc[i][1:4]
                        break
                    else:
                        mail = mail + str(df.loc[i][0])

            j = j + 1
      else:
            print('else ', i)
            if df.loc[i][0]!='':
                dfpr.loc[j] = df.loc[i]
                j = j + 1
    print(dfpr)
    #dfpr = dfpr.dropna()
    df = dfpr
    return df

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe("language_detector")
def lang_detecton(text):
    doc = nlp(text)
    return(doc._.language)


def input_Clean(subject,textbody):
  # nlp.add_pipe(nlp.create_pipe('sentencizer'))
  salutation1 = [
        'dear',
        'hello',
        'good day',
        'pls',
        'please',
        'good afternoon,',
        're:',
        ]
  salutation2 = ['maersk', 'team', 'sir/madam', 'greetings', 'sir / mam']
  headers = [
        'sent:',
        'http',
        'https',
        'www',
        'subject:',
        'pop/imap mail',
        '.png',
        '.jpg',
        '.jpeg',
        'important notice',
        'notice',
        'note:',
        'forwarded message',
        'original message',
        'importance:',
        'caution:',
        'ltd',
        ]
  disclaimer = [
        'outlook mobile',
        'my iPhone',
        'yahoo mail',
        'if you have received this communication in error, notify the sender by email and erase this communication immediately.'
            ,
        'the above message is intended solely for the named addressee.',
        'message was sent from outside of your organization',
        'if you have received this in error,   notify us immediately by return email, or telephone (call us collect) and delete this email.'
            ,
        'further, what is expressed in this email are the views of the individual sender(s) and may not reflect the views of the silver fern farms group.'
            ,
        'process your personal data',
        'image removed by sender',
        'this message was sent by',
        'consider the environment before printing this email',
        'please do not reply to this message',
        'you are not the intended recipient',
        'confidential and / or privileged material intent solely for the recipient'
            ,
        'named in this message. If the reader of this message in not the intended'
            ,
        'recipient you are notified that any use, disclosure, copying or distribution'
            ,
        'of this information is prohibited. if you have received this message in'
            ,
        'if you are not satisfied with the quality of service provided, you may'
            ,
        'error please notify us immediately.',
        'we do not give or endorse any opinions, conclusions and other information in this message that do not relate to our official business.'
            ,
        'the information contained in this communication (including any attachments) is privileged and confidential, and may be legally exempt from disclosure under applicable law.'
            ,
        'ft is intended only for the specific purpose of being used by the individual or entity to whom it is addressed.'
            ,
        'please also ensure that this original message and any record of it is permanently deleted from your computer system.'
            ,
        ]
  signatures = [
        'best regards',
        'regards',
        'kind regards',
        'ltd',
        'rgds',
        'rgd',
        'regds',
        'best regard',
        '----',
        '____',
        'warm regards',
        "your's truly",
        "your's kindly",
        "your's sincerely",
        ]


  #textbody = quotations.extract_from(textbody, 'text/plain')
  textbody = quotations.extract_from_plain(textbody)
  textbody=re.sub(r'\n+', ' ', textbody).strip()
  textbody=re.sub('\n',' ',textbody) 
  #print(textbody) 
  input_string = ". ".join([subject,textbody])                      #concatenate subject and mailbody
  doc = nlp(input_string)
  from_flag=0                                                         #flag to extract latest mail from a mail trail
  lst_english_sentences=[]
  cleansed_list=[]
  for sent in doc.sents:
    if ((lang_detecton(sent.text)=='en') and len(sent.text)!=0):    #spacy to split input text and extract ony non empty English text
      lst_english_sentences.append(sent.text)                      
    else:
      continue
    #remove non ascii characters
  only_ascii_en_char=[]
  for string in lst_english_sentences:
   for c in string:
      if ord(c) > 127:
         string=string.replace(c," ")
      else:
        continue
   only_ascii_en_char.append(string)
 
 
  #salutation removal
  #print(only_ascii_en_char)
  for salute in salutation1:
      #print(k)
      #print("salutation1 check")
      i=0
      for sentence in only_ascii_en_char:
        
        #print(sentence)
        #print(salute)
        if salute in sentence.lower():
          #print("salute is in sentence")
          #print(sentence)
          sentence_new = sentence.lower().replace(salute,'')
          only_ascii_en_char[i]=sentence_new
          #print(sentence)
          i=i+1
        else:
          i=i+1
  for salute in salutation2:
      #print(k)
      #print("salutation1 check")
      i=0
      for sentence in only_ascii_en_char:
        #print(sentence)
        #print(salute)
        if salute in sentence.lower():
          #print("salute is in sentence")
          #print(sentence)
          sentence_new = sentence.lower().replace(salute,'')
          only_ascii_en_char[i]=sentence_new
          #print(sentence)
          i=i+1
        else:
          i=i+1

  #signature removal
  for sign in signatures:
    #print(k)
    #print("salutation1 check")
    i=0
    for sentence in only_ascii_en_char:
      #print(sentence)
      #print(salute)
      if sign in sentence.lower():
        #print("salute is in signatures")
        #print(sentence)  
        #print(i)     
        only_ascii_en_char=only_ascii_en_char[:i]
        i=i+1 
        #print(sentence)
    
      else:
        i=i+1
    
    #input text cleaning
  i=0
  #print(only_ascii_en_char)
  while i<=len(only_ascii_en_char)-1:
   
    #print(only_ascii_en_char[i])
    
     #disclaimer removal
    if only_ascii_en_char[i].lower() in disclaimer:
      i=i+1
      #print("inside disclaimer")
      continue

    if only_ascii_en_char[i].lower() in signatures:
      i=i+1
      #print("inside signature")
      continue
    
    if only_ascii_en_char[i].lower() in headers:
      i=i+1
      #print("inside headers")
      continue

    #extract latest mail only
   

  
    #print(only_ascii_en_char[i])
    cleansed_list.append(only_ascii_en_char[i])

    #print("here")
    i=i+1
  
  cleansed_string = ' '.join(cleansed_list)

  #checking length of final string
  
  cleansed_string=re.sub(' +', ' ', cleansed_string)      ##removing multiple spaces with a singe space
  if len(cleansed_string) > 300:
    cleansed_string1=cleansed_string[0:300]
    cleansed_string2=cleansed_string[301:]
    doc1 = nlp(cleansed_string2)
    for sent in doc1.sents:
      cleansed_string1="".join([cleansed_string1,sent.text])
      break
    if len(cleansed_string1)>30:
      return (cleansed_string1)
    else:
      return "Cleansed data is less than 30 characters"
  else:
    if len(cleansed_string)>30:
      return (cleansed_string)
    else:
      return "Cleansed data is less than 30 characters"


time: 1.21 s (started: 2021-09-28 16:58:19 +00:00)


In [21]:
x = """this is a line








this i new line 
"""


re.sub(r'\n+', ' ', x).strip()

'this is a line this i new line'

time: 8.13 ms (started: 2021-09-28 19:21:53 +00:00)


In [13]:
lang_detecton('testing lang')

'en'

time: 39.3 ms (started: 2021-09-28 16:57:16 +00:00)


In [7]:
f = '/content/drive/MyDrive/raw_train_data.csv'
df=pd.DataFrame()
df = read_csv_file1(f)
df = df.dropna()
df0 = msg_parse(df)

Streaming output truncated to the last 5000 lines.
else  337504
else  337505
else  337506
else  337507
else  337508
else  337509
else  337510
else  337511
else  337512
else  337513
else  337514
else  337515
else  337516
else  337517
else  337518
else  337519
else  337520
else  337521
else  337522
else  337523
else  337524
else  337525
else  337526
else  337527
else  337528
else  337529
else  337530
else  337531
else  337532
else  337533
else  337534
else  337535
else  337536
else  337537
else  337538
else  337539
else  337540
else  337541
else  337542
else  337543
else  337544
else  337545
else  337546
else  337547
else  337548
else  337549
else  337550
else  337551
else  337552
else  337553
else  337554
else  337555
else  337556
else  337557
else  337558
else  337559
else  337560
else  337561
else  337562
else  337563
else  337564
else  337565
else  337566
else  337567
else  337568
else  337569
else  337570
else  337571
else  337572
else  337573
else  337574
else  337575
else  337576


In [8]:
df0.shape

(23756, 8)

time: 4.22 ms (started: 2021-09-28 16:53:14 +00:00)


In [9]:
import fasttext

fasttext.FastText.eprint = lambda x: None

time: 1.9 ms (started: 2021-09-28 16:53:14 +00:00)


In [16]:
lst=[]
from datetime import datetime
start = datetime.now()
try:
  for i in range(df0.shape[0]):
    
    if (i%100==0):
      end = datetime.now()
      time_taken = end - start
      print('Time: ',time_taken) 
      print("Records processed:",i,"  of ",df0.shape[0])
      start = datetime.now()
    lst.append(input_Clean(df0['Subject'].values[i],df0['TextBody'].values[i]))
#print(str1)
except Exception as err:
  print(err)
finally:
    #pre_processed.to_excel('Cleaned_data_Experiment.xlsx')
  pre_processed = pd.DataFrame(lst,columns=['Cleansed_data'])
  pre_processed['Text Body']=df0['TextBody']
  pre_processed['Sub Type']=df0['sub_type__c']
  pre_processed['Case Number']=df0['CaseNumber']
  pre_processed.to_excel('Cleaned_data_Experiment1.xlsx')



Time:  0:00:00.000201
Records processed: 0   of  23756
Time:  0:00:13.299489
Records processed: 100   of  23756
Time:  0:00:16.918295
Records processed: 200   of  23756
Time:  0:00:10.984782
Records processed: 300   of  23756
Time:  0:00:13.112679
Records processed: 400   of  23756
Time:  0:00:13.114428
Records processed: 500   of  23756
Time:  0:00:12.073612
Records processed: 600   of  23756
Time:  0:00:10.720972
Records processed: 700   of  23756
Time:  0:00:12.471580
Records processed: 800   of  23756
Time:  0:00:03.933633
Records processed: 900   of  23756
Time:  0:00:01.241271
Records processed: 1000   of  23756
Time:  0:00:11.004292
Records processed: 1100   of  23756
Time:  0:00:14.630556
Records processed: 1200   of  23756
Time:  0:00:12.311444
Records processed: 1300   of  23756
Time:  0:00:10.844619
Records processed: 1400   of  23756
Time:  0:00:12.491924
Records processed: 1500   of  23756
Time:  0:00:14.819579
Records processed: 1600   of  23756
Time:  0:00:11.390710
Reco

IllegalCharacterError: ignored

time: 7min 28s (started: 2021-09-28 16:58:25 +00:00)


In [ ]:
pre_processed.to_csv('/content/drive/new_training_data.csv',index=False)